<a href="https://colab.research.google.com/github/fakhrulfaiz/hands_on_machine_learning/blob/main/Titanic_survival_classiffier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_df.shape

(891, 12)

In [ ]:
train_data = train_df.set_index("PassengerId")
test_data = test_df.set_index("PassengerId")

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
train_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
        ("ordinal_encoder", OrdinalEncoder()),
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output=False)),
    ])

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
X_train = train_data.drop("Survived", axis=1)
y_train = train_data["Survived"]

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
import numpy as np

classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(random_state=42),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}


results = {}

print("Evaluating classifiers using 5-fold cross-validation...\n")
print(f"{'Classifier':<25} {'Mean Accuracy':<15} {'Std Dev':<10}")
print("=" * 50)

# Test each classifier
for name, classifier in classifiers.items():
    # Create full pipeline with preprocessing + classifier
    full_pipeline = Pipeline([
        ("preprocess", preprocess_pipeline),
        ("classifier", classifier)
    ])

    scores = cross_val_score(full_pipeline, X_train, y_train,
                            cv=5, scoring='accuracy')

    results[name] = {
        'mean': scores.mean(),
        'std': scores.std(),
        'scores': scores
    }

    print(f"{name:<25} {scores.mean():.4f}          {scores.std():.4f}")


best_classifier = max(results, key=lambda x: results[x]['mean'])

print("\n" + "=" * 50)
print(f"🏆 Best Classifier: {best_classifier}")
print(f"   Mean Accuracy: {results[best_classifier]['mean']:.4f} (+/- {results[best_classifier]['std']:.4f})")
print(f"   Individual fold scores: {results[best_classifier]['scores']}")


Evaluating classifiers using 5-fold cross-validation...

Classifier                Mean Accuracy   Std Dev   
Logistic Regression       0.7901          0.0216
Decision Tree             0.7890          0.0263
Random Forest             0.8048          0.0314
Gradient Boosting         0.8283          0.0217
SVM                       0.8272          0.0168
KNN                       0.8059          0.0250
Naive Bayes               0.7834          0.0257

🏆 Best Classifier: Gradient Boosting
   Mean Accuracy: 0.8283 (+/- 0.0217)
   Individual fold scores: [0.82122905 0.80898876 0.85393258 0.80337079 0.85393258]


In [ ]:
from sklearn.model_selection import cross_val_score

# Test Gradient Boosting with multiple metrics
rf_pipeline = Pipeline([
    ("preprocess", preprocess_pipeline),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])

print("Gradient Boosting - Detailed Metrics:")
print(f"Accuracy:  {cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='accuracy').mean():.4f}")
print(f"Precision: {cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='precision').mean():.4f}")
print(f"Recall:    {cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='recall').mean():.4f}")
print(f"F1-Score:  {cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='f1').mean():.4f}")

Gradient Boosting - Detailed Metrics:
Accuracy:  0.8048
Precision: 0.7547
Recall:    0.7279
F1-Score:  0.7395


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(rf_pipeline, param_grid, cv=5,
                           scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print(f"Best params: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best params: {'classifier__min_samples_split': 5, 'classifier__n_estimators': 300}
Best score: 0.8294


In [ ]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:

passenger_ids = test_df["PassengerId"].copy()

best_model = grid_search.best_estimator_

predictions = best_model.predict(test_data)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})

In [ ]:

submission.to_csv('submission.csv', index=False)


# SVM

In [ ]:
svm_pipeline = Pipeline([
    ("preprocess", preprocess_pipeline),
    ("classifier", SVC(random_state=42))
])

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__C': [0.1, 1, 10, 100],                    # Regularization parameter
    'classifier__kernel': ['linear', 'rbf', 'poly'],       # Kernel type
    'classifier__gamma': ['scale', 'auto', 0.001, 0.01, 0.1],  # Kernel coefficient
    'classifier__degree': [2, 3, 4]                        # Degree for poly kernel only
}

grid_search = GridSearchCV(svm_pipeline, param_grid, cv=5,
                           scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print(f"Best params: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

Fitting 5 folds for each of 180 candidates, totalling 900 fits
Best params: {'classifier__C': 10, 'classifier__degree': 4, 'classifier__gamma': 'auto', 'classifier__kernel': 'poly'}
Best score: 0.8283


In [ ]:

passenger_ids = test_df["PassengerId"].copy()

best_model = grid_search.best_estimator_

predictions = best_model.predict(test_data)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})

In [ ]:
submission.to_csv('submission.csv', index=False)
